# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [1]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

C:\Users\emigi\AppData\Local\Temp\ipykernel_22400\3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [2]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",

    # New patterns to remove dotted abbreviations
    r"\bs\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"\bs\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"\bs\.?a\.?\b": "",      # Handles "S.A."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [3]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"\s*\.+\s*", " ", name) 
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [4]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [5]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 37656 su 76808 (49.03%)


,company_name,normalized_name
7049,bad and boujee nail salon ltd,bad and boujee nail salon
3717,"amazon.com.dedc, llc",amazon com dedc
73350,wingtech technology,wingtech
20045,ebro foods sa,ebro foods
34042,isuzu motors ltd,isuzu motors
76702,NaN,
34172,itochu corp,itochu
59170,showa denko materials co ltd,showa denko materials
28937,h. lundbeck as,h lundbeck
40468,marcus & millichap inc,marcus millichap


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.



In [6]:
companies_df['company_name'] = companies_df['normalized_name']
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.to_csv('../aziende_normalizzate.csv', index=False)

# **Normalizzazione delle città**

In [7]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
34782,jetti resources,boulder
72722,wcs limited,daventry
41645,mercury 1,roma
5749,aspen,bedford
57841,segmueller vertriebs,bolzano
28946,h j heinz,hayes
47295,onyx graphics,midvale
47359,opis,desio
23281,fiesta real estate,tallinn
54174,raymond gubbay limited,london


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [8]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

In [9]:
filtered_df['city'] = filtered_df['city'].apply(clean_city)
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
71609,videndum media,cassola
21532,epic! creations,redwood city
72817,weee!,fremont
5749,aspen,bedford
21813,esl gaming america,burbank
46528,nuveen italy,milano
46386,ntt limited,london
40512,marini,alfonsine
21903,essilor of america,dallas
43773,nadella,milano


In [10]:
# Apply the changes to the df
companies_df['city'] = companies_df['city'].apply(clean_city)

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [11]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [14]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
562,aberdeen standard investments ireland limited,milano
622,above comparison,milano
623,above comparison,milano
766,accenture,milano
765,accenture,milano
...,...,...
73454,withsecure,milano
74336,yes ticket,milano
74335,yes ticket,milano
75694,zurich italy bank,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [15]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
1431,aec underwriting agenzia di assicurazione e ri...,roma
1430,aec underwriting agenzia di assicurazione e ri...,roma
1739,agenzia delle entrate,roma
2463,ald automotive italia,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
...,...,...
68023,trenitalia,roma
69746,universita' degli studi di roma la sapienza,roma
69748,universita' degli studi roma tre,roma
73045,western union payment ireland limited,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [17]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_normalizer")
city_cache = {}

In [19]:
def get_city_in_italian(city):
    if pd.isna(city) or not isinstance(city, str) or city == "":
        return ""

    if city in city_cache:
        print(f"Cache hit! {city}")
        return city_cache[city]
    
    try:
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                print(f"translating {location_it}...")
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [20]:
def apply_geocoding(city):
    result = get_city_in_italian(city)
    time.sleep(1)
    return result

In [21]:
milan_df['city'] = milan_df['city'].apply(apply_geocoding)
display(milan_df[['company_name', 'city']])

translating Milano, Lombardia, Italia...
Cache hit! milan


C:\Users\emigi\AppData\Local\Temp\ipykernel_22400\1402383521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milan_df['city'] = milan_df['city'].apply(apply_geocoding)


,company_name,city
57140,satispay,milano
57364,scalapay,milano


**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [23]:
companies_df['city_translate'] = companies_df.apply(
    lambda row: apply_geocoding(row['city']), 
    axis=1
)
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

KeyboardInterrupt: 

In [58]:
num_modified = companies_df['modified'].sum()
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = companies_df[companies_df['modified']][['company_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['company_name'])
companies_df[['company_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 37656 su 76808 (49.03%)


KeyError: "['city_translate'] not in index"

In [24]:
filtered_df = companies_df[companies_df['country'].notna() & (companies_df['country'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
21737,ernest jackson limited,united kingdom
15176,cnh u k limited,united kingdom
39687,l'exception,france
57705,sealed air,usa
36762,kla,usa
48730,paddle,united kingdom
24644,forsyth barnes,uk
57034,sapient limited,united kingdom
15167,cnh industrial italia,it00370290363
14219,china-singapore suzhou industrial park develop...,china


In [28]:
country_mapping = {
    'it': 'italy',            # Codici e abbreviazioni per l'Italia
    'it02968430237': 'italy', # Caso specifico con un codice fiscale italiano
    'uk': 'united kingdom',   # Abbreviazione per il Regno Unito
    'usa': 'united states',   # Abbreviazione per gli Stati Uniti
    'us': 'united states',    # Abbreviazione per gli Stati Uniti (altro formato)
    'fr': 'france',           # Abbreviazione per la Francia
    'de': 'germany',          # Abbreviazione per la Germania
    'es': 'spain',            # Abbreviazione per la Spagna
    'ca': 'canada',           # Abbreviazione per il Canada
    'au': 'australia',        # Abbreviazione per l'Australia
    'cn': 'china',            # Abbreviazione per la Cina
    'in': 'india',            # Abbreviazione per l'India
    'jp': 'japan',            # Abbreviazione per il Giappone
    'br': 'brazil',           # Abbreviazione per il Brasile
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'mx': 'mexico',           # Abbreviazione per il Messico
    'se': 'sweden',           # Abbreviazione per la Svezia
    'no': 'norway',           # Abbreviazione per la Norvegia
    'fi': 'finland',          # Abbreviazione per la Finlandia
    'nl': 'netherlands',      # Abbreviazione per i Paesi Bassi
    'ch': 'switzerland',      # Abbreviazione per la Svizzera
    'ru': 'russia',           # Abbreviazione per la Russia
    'sa': 'south africa',     # Abbreviazione per il Sudafrica
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'sg': 'singapore'         # Abbreviazione per Singapore
}

In [32]:
import re

# Nuovo dizionario di sostituzione per i codici che iniziano con "it" seguiti da numeri
CODE_REPLACEMENTS = {
    r"\bit\d+\b": "italy"  # Codici fiscali italiani che iniziano con "it" seguito da numeri (lunghezza variabile)
}

# Funzione per normalizzare il paese usando il dizionario
def normalize_country(country):
    # Rimuove gli spazi, converte tutto in minuscolo e cerca nel dizionario
    country = country.strip().lower()
    return country_mapping.get(country, country)  # Restituisce il nome completo o il valore originale

# Applicare la normalizzazione della colonna 'country'
filtered_df['normalized_country'] = filtered_df['country'].apply(normalize_country)

# Visualizzare una selezione casuale delle righe per verificare
display(filtered_df[['company_name', 'country', 'normalized_country']].sample(25))

,company_name,country,normalized_country
9029,beijing aosaikang pharmaceutical,china,china
11870,calpeda,italy,italy
958,actavis laboratories ut,united states,united states
12202,capitalonline data service,china,china
51013,predictive,thailand,thailand
60452,snam,italy,italy
73127,wha premium growth freehold and leasehold real...,thailand,thailand
45846,nohup,italy,italy
51512,proto labs,usa,united states
58694,sheela foam,india,india
